In [1]:
!pip install opencv-python
!pip install opencv-python-headless

     |████████████████████████████████| 60.9 MB 31.9 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 48.3 MB 21.7 MB/s eta 0:00:01     |██████████████████▊             | 28.3 MB 21.7 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
pip install -U scikit-image

     |████████████████████████████████| 12.4 MB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 148 kB 90.6 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 55.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 45.6 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import scipy

In [4]:
pip install -U scikit-learn

     |████████████████████████████████| 22.2 MB 29.2 MB/s eta 0:00:01
     |████████████████████████████████| 309 kB 87.7 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16 as vgg16
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 as mobilenetv2
from tensorflow.keras.applications.resnet50 import ResNet50 as resnet50
from tensorflow.keras.applications.inception_v3 import InceptionV3 as inception
from tensorflow.keras.applications.densenet import DenseNet169 as densenet
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Model
from keras import layers, Sequential
from matplotlib import pyplot
from math import floor
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
from numpy import expand_dims, array, exp, max
import json
import skimage.feature as feature
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.feature import local_binary_pattern
import sklearn
from sklearn.decomposition import PCA

from skimage.transform import resize

In [19]:
model1 = densenet()
model2 = Model(inputs=model1.inputs, outputs=model1.layers[2].output)
# model.add(layers.MaxPool2D(pool_size=(2, 2),strides=(1, 1), padding='valid'))
model = Sequential(layers=model2.layers)
model.add(layers.MaxPool2D(pool_size=(3, 3), strides=(3, 3), padding='same'))
# model = Model(inputs=model.inputs, outputs = model.layers[1].output)

model.summary()

58548224/58541896 [==============================] - 0s 0us/step
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 230, 230, 3)       0         
_________________________________________________________________
conv1/conv (Conv2D)          (None, 112, 112, 64)      9408      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 38, 38, 64)        0         
Total params: 9,408
Trainable params: 9,408
Non-trainable params: 0
_________________________________________________________________


In [20]:
class LocalBinaryPatterns:
  def __init__(self, numPoints, radius):
    self.numPoints = numPoints
    self.radius = radius

  def describe(self, image, eps = 1e-7):
    lbp = feature.local_binary_pattern(image, self.numPoints, self.radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, self.numPoints+3), range=(0, self.numPoints + 2))

    # Normalize the histogram
    hist = hist.astype('float')
    hist /= (hist.sum() + eps)

    return hist, lbp

In [21]:
###########################THIS CODE CHECKS IMAGES FROM A FOLDER#################################
def sortTrainImages():
    # get the class labels from training datasets
    p = {}
    data_path = '../../../paper3/Caltech256/caltech256data/256_ObjectCategories'
    img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255, rotation_range=20)
    labels = img_gen.flow_from_directory(data_path)
    train_labels = labels.class_indices.keys()

    for lbl in train_labels:
        resnet_path = data_path +'/' + lbl
        p_files = [f for f in listdir(resnet_path) if isfile(join(resnet_path, f))]
        pests_images = np.empty(len(p_files), dtype=object)

        for m in range(0, len(p_files)):
            imageName_x = p_files[m]
            #p.update({lbl + "_" + str(m): imageName_x})
            #print(join(resnet_path, p_files[m]))
            if m < 30:                
                # get the name of the image
                imageName_x = p_files[m]
                p.update({lbl + "_" + str(m): imageName_x})

    return p


images_collection = sortTrainImages()

Found 30607 images belonging to 257 classes.


In [9]:
print(images_collection)

{'001.ak47_0': '001_0007.jpg', '001.ak47_1': '001_0091.jpg', '001.ak47_2': '001_0025.jpg', '001.ak47_3': '001_0090.jpg', '001.ak47_4': '001_0087.jpg', '001.ak47_5': '001_0014.jpg', '001.ak47_6': '001_0065.jpg', '001.ak47_7': '001_0073.jpg', '001.ak47_8': '001_0088.jpg', '001.ak47_9': '001_0026.jpg', '001.ak47_10': '001_0049.jpg', '001.ak47_11': '001_0043.jpg', '001.ak47_12': '001_0082.jpg', '001.ak47_13': '001_0022.jpg', '001.ak47_14': '001_0012.jpg', '001.ak47_15': '001_0033.jpg', '001.ak47_16': '001_0080.jpg', '001.ak47_17': '001_0006.jpg', '001.ak47_18': '001_0074.jpg', '001.ak47_19': '001_0083.jpg', '001.ak47_20': '001_0056.jpg', '001.ak47_21': '001_0036.jpg', '001.ak47_22': '001_0010.jpg', '001.ak47_23': '001_0072.jpg', '001.ak47_24': '001_0089.jpg', '001.ak47_25': '001_0047.jpg', '001.ak47_26': '001_0041.jpg', '001.ak47_27': '001_0023.jpg', '001.ak47_28': '001_0093.jpg', '001.ak47_29': '001_0057.jpg', '001.ak47_30': '001_0075.jpg', '001.ak47_31': '001_0060.jpg', '001.ak47_32': '0

In [22]:
def softMax(temperature, features_list):
    feat_list = [x / temperature for x in features_list]
    feature_list = exp(feat_list - max(feat_list))
    return feature_list / feature_list.sum(axis=0)

In [23]:
#for testing scikit learn lbp implementation
# type of LBP
METHOD = 'uniform'  # at most two circular 0-1 or 1-0 transitions
radius = 1  # distance between central pixels and comparison pixels
n_points = 8 * radius  # define number of comparison pixels

#all features
all_features= []


In [ ]:
#second method
for o, j in images_collection.items():
    # create list for LBP representations
    lbp_imgs = []
    
    o = o.split('_')
    o = o[0]
    #print("class:", o)
    image_string = '../../../paper3/Caltech256/caltech256data/256_ObjectCategories/' + o + '/' + j
    
    img = imread(image_string)
    img = resize(img,(224,224,3))
    # get the path of image to extract the class
    img_class = o
    img = img_to_array(img)    
    # expand the image dimensions
    img = expand_dims(img, axis=0)
    print("img:")
    # scale the pixels
    img = preprocess_input(img)
    # get the features of the first layer
    feature_maps = model.predict(img)
   
    ix = 1
    square = 3
    features_prob_dist_array = []
    
    
    for _ in range(square):     
        
        each_main_feature = []
        
        for _ in range(square):
            ax = pyplot.subplot(square, square, ix)
            ax.set_xticks([])
            ax.set_yticks([])
            ##################FLATTEN THE FEATURE for dimensionality reduction##################################
            feature_items = array(feature_maps[0, :, :, ix - 1])
            #get the unsigned values of the feature
            features_x = feature_items.astype(np.uint8)
            lbp = local_binary_pattern(features_x, n_points, radius, METHOD)
            
            each_feature = []
            
            for h in lbp:
                #get the top 5 important features
                n_array = np.array(h)
                #each_sub_feature = np.prod(h, axis = 0, where = h > 0, keepdims = True)
                n_array= np.sort(n_array)
                each_soft = softMax(0.5,n_array[-8:])
                prod_sub_items = np.prod(each_soft)
                each_feature.append(prod_sub_items)
            numerator = each_feature
            denominator = np.trapz(numerator,axis=0)
            conf_list =(numerator/denominator)
            
            #final_array = softMax(0.5,features_prob_dist_array), the list is based on the softmax prob
            final_array_numerator=  (softMax(0.5,conf_list))
            final_array_denominator = np.trapz(final_array_numerator,axis=0)
            
            
            final_array_conf_list = final_array_numerator/final_array_denominator
            
            each_item_x = np.prod(final_array_conf_list)
            
            each_main_feature.append(each_item_x)
        
            #final_array_numerator=str(final_array_numerator)
            #first_chars = float(final_array_numerator[0:8])
            #print(first_chars)
            
            ix += 1
        #print(each_main_feature)   
        #get the conflation of all the features of the specific image
        
    #print(type(each_main_feature))
    
    new_rec = {j + "_/" + o: each_main_feature}
    
    with open("DenseNet169/caltech_target_domain_lbp.json", 'r+') as file:
            # First we load existing data into a dict.
            file_data = json.load(file)
            # Join new_data with file_data inside emp_details
            file_data["target_images"].append(new_rec)
            # Sets file's current position at offset.
            file.seek(0)
            # convert back to json.
            json.dump(file_data, file, indent=4)
    print("*")
       
    
       
        
    

img:
*
img:
*
img:
*
img:
*
img:


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:

*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:

*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:

*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:

*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:

*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:
*
img:

In [15]:
for m in all_features:
    #for each image
        main_feature =[]

        for p in m:
            feature = []
            for d in p:

                each_item = np.prod(d, axis = 0, where = d > 0, keepdims = True)
                #convert eeach to softmax equivalent, get the top 8 features
                feature.append(each_item[0])           
            #sorted_feature = feature.sort()
            sorted_feature= np.sort(feature)

            each_soft = softMax(0.1,sorted_feature[-10:]) 
            #do another np on eachsoft
            t = np.prod(each_soft, axis = 0, where = each_soft > 0, keepdims = True)
            val_r = format(t[0], ".4f")
            main_feature.append(float(val_r))

        x = np.prod(main_feature, axis = 0, keepdims = True)
        numerator = main_feature
        denominator = np.trapz(numerator,axis=0)
        image_conf_list = numerator/denominator

        print(image_conf_list)

In [81]:
one_feature_item = []
for m in all_features:
    #for each image
    main_feature =[]
    
    for p in m:
        feature = []
        for d in p:
            
            each_item = np.prod(d, axis = 0, where = d > 0, keepdims = True)
            #convert eeach to softmax equivalent, get the top 8 features
            feature.append(each_item[0])           
        #sorted_feature = feature.sort()
        sorted_feature= np.sort(feature)
        
        each_soft = softMax(0.1,sorted_feature[-10:]) 
        #do another np on eachsoft
        t = np.prod(each_soft, axis = 0, where = each_soft > 0, keepdims = True)
        val_r = format(t[0], ".4f")
        main_feature.append(float(val_r))
        
    x = np.prod(main_feature, axis = 0, keepdims = True)
    numerator = main_feature
    denominator = np.trapz(numerator,axis=0)
    image_conf_list = numerator/denominator
    
    print(image_conf_list)
    #one_feature_item.append(x[0])
#do the trapz to get the denominator
#denom = np.trapz(one_feature_item,axis=0)


#print(conf_list)

[0.1537539  0.1537539  0.1537539  0.1537539  0.1537539  0.00059964
 0.1537539  0.1537539  0.        ]
[0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
[0.18175374 0.         0.18175374 0.18175374 0.18175374 0.
 0.18175374 0.18175374 0.00070884]
[0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
[0.15905837 0.03976459 0.00588516 0.15905837 0.15905837 0.15905837
 0.15905837 0.15905837 0.15905837]
[0.17777778 0.         0.17777778 0.17777778 0.17777778 0.
 0.17777778 0.17777778 0.04444444]
[0.13114754 0.13114754 0.13114754 0.13114754 0.13114754 0.13114754
 0.13114754 0.13114754 0.03278689]
[0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
[0.1537539  0.1537539  0.1537539  0.1537539  0.1537539  0.00059964
 0.1537539  0.1537539  0.        ]
[0.1537539  0.1537539  0.1537539  0.1537539  0.1537539  0.00059964
 0.1537539  0.1537539  0.        ]
[0.1537539  0.1537539  0.1537539  0.1537539  0.1537539  0.00059964
 0.1537539  0.1537539  0.        ]
[0.125 0.125 0.125 0.125 0.125 0

In [ ]:
_maps[0, :, :, ix - 1])
                
            ix += 1
    break

In [ ]:
    prod_dists = []
    for feat in features_prob_dist_array:
        print(feat.shape)

In [ ]:

    print("Image:", image_string)
    
    for _ in range(square):
        for _ in range(square):
            ax = pyplot.subplot(square, square, ix)
            ax.set_xticks([])
            ax.set_yticks([])
            ##################FLATTEN THE FEATURE for dimensionality reduction##################################
            feature_items = array(feature_maps[0, :, :, ix - 1]).flatten()
            #print(feature_items)
            # convert all the values in the feature map into probability distributions
            #soft_maxed = softMax(temp, feature_items)            
            #num_of_non_zeros = np.count_nonzero(soft_maxed)
            # check if the feature does not contain zeros; if not append the feature
            #if num_of_non_zeros != 0:                
                #print("All-", len(soft_maxed), 'non-zeros-',num_of_non_zeros)
            features_prob_dist_array.append(feature_items)
            pyplot.imshow(feature_maps[0, :, :, ix - 1], cmap='gray')
                
            ix += 1
    m = lambda m: np.prod(m, axis = 0)
    
    list_1 = floor(len(features_prob_dist_array) / 2)
    list_2 = len(features_prob_dist_array) - list_1     
    new_array = features_prob_dist_array[0:list_1]
    new_array2 = features_prob_dist_array[list_1:len(features_prob_dist_array)]
    
    new_final_array = []
    
    for da in new_array:
        
        my_xx = da[da != 0]
        try:
            t = softMax(0.5,my_xx)
            new_final_array.append(t)
        except ValueError:  #raised if `y` is empty.
            pass
        
        
        
    for da in new_array2:       
        my_xx = da[da != 0]
        try:
            t = softMax(0.5,my_xx)
            new_final_array.append(t)
        except ValueError:  #raised if `y` is empty.
            pass
    
    other_final = []
    for i in new_final_array:
        i_ = i[i != 0]
        other_final.append(i_)
       
    numerator = []
    for jp in other_final:
        my_list = np.sort(jp, axis=None)
        #get the most important values
        numerator.append(np.prod(my_list[-3:]))

    
    denominator = np.trapz(numerator,axis=0)
    
    sum_yote1 = 0
    if denominator !=0:

        conflated_distribution = numerator / denominator
        #print("conflated distribution is:")
        #print(conflated_distribution)
        for f in range(0, len(conflated_distribution)):
            sum_yote1 += conflated_distribution[f]
        #print("sum is:", sum_yote1)

        new_rec = {j + "_/" + o: conflated_distribution.tolist()}
        
        with open("files/source_domain_data.json", 'r+') as file:
            # First we load existing data into a dict.
            file_data = json.load(file)
            # Join new_data with file_data inside emp_details
            file_data["source_images"].append(new_rec)
            # Sets file's current position at offset.
            file.seek(0)
            # convert back to json.
            json.dump(file_data, file, indent=4)
        #print("IMAGE:", j, 'FOR CLASS:',o)
    # break;